# <center> ML-7. Оптимизация гиперпараметров модели

In [67]:
#Импорты

import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные модели
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn.model_selection import train_test_split #сплитование выборки
from sklearn.model_selection import cross_val_score #кросс-валидация

# Импорт оптимизаторов
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from hyperopt import hp, fmin, STATUS_OK, tpe, Trials
import optuna

# В процессе выполнения работы было много различных 
# предупреждений, многие исправил, но остались в основном 
# те, которые говорят о недостаточном ранжировании 'C' 
# для чистоты выводов я их уберу.
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
plt.style.use('seaborn-v0_8')

In [68]:
data = pd.read_csv('data/_train_sem09.csv')

data

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3746,1,0.033300,0.506409,0.10,0.0,0.209887,0.633426,0.297659,0.376124,0.727093,...,0,0,0,0,0,0,0,0,0,0
3747,1,0.133333,0.651023,0.15,0.0,0.151154,0.766505,0.170876,0.404546,0.787935,...,0,0,1,0,1,0,1,0,0,0
3748,0,0.200000,0.520564,0.00,0.0,0.179949,0.768785,0.177341,0.471179,0.872241,...,0,0,0,0,0,0,0,0,0,0
3749,1,0.100000,0.765646,0.00,0.0,0.536954,0.634936,0.342713,0.447162,0.672689,...,0,0,0,0,0,0,0,0,0,0


In [69]:
# Проверяем, есть ли пропуски в любом из столбцов
has_missing = data.isnull().any().any()

if has_missing:
    print("В датафрейме есть пропуски.")
else:
    print("В датафрейме нет пропусков.")

В датафрейме нет пропусков.


In [70]:
#Смотрим распреление значений целевого признака
data['Activity'].value_counts(normalize=True)

Activity
1    0.542255
0    0.457745
Name: proportion, dtype: float64

Разбиение почти ровное, но стратификация, думаю, не повредит.

In [71]:
# Разделение данных на выборки

X = data.drop(['Activity'], axis=1)
y = data['Activity']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)

In [72]:
#Построение изначальных базовых моделей
print('Логистическая регрессия:')
lr_base = linear_model.LogisticRegression(random_state=42, max_iter=1200)

lr_base.fit(X_train, y_train)
print("accuracy на тестовом наборе: {:.2f}".format(lr_base.score(X_test, y_test)))
y_test_pred = lr_base.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))


print('-'*50)
print('Случайный лес:')
rf_base = ensemble.RandomForestClassifier(random_state=42)

rf_base.fit(X_train, y_train)
print("accuracy на тестовом наборе: {:.2f}".format(rf_base.score(X_test, y_test)))
y_test_pred = rf_base.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Логистическая регрессия:
accuracy на тестовом наборе: 0.75
f1_score на тестовом наборе: 0.78
--------------------------------------------------
Случайный лес:
accuracy на тестовом наборе: 0.80
f1_score на тестовом наборе: 0.82


Базовые митрики есть. Приступаем к оптимизации гиперпараметров.

## <center> Grid Search

In [73]:
#Линейная регрессия
param_grid_lr = [
              {'penalty': ['l2', None] , # тип регуляризации
              'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регурялизации
              
              {'penalty': ['l1', 'l2'] ,
              'solver': ['liblinear', 'saga'],
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}
]

grid_search_lr = GridSearchCV(
    estimator=linear_model.LogisticRegression(
        random_state=42, #генератор случайных чисел
        max_iter=1200 #количество итераций на сходимость
    ), 
    param_grid=param_grid_lr, 
    cv=5, 
    n_jobs = -1
)  

%time grid_search_lr.fit(X_train, y_train) 
print("accuracy на тестовом наборе: {:.2f}".format(grid_search_lr.score(X_test, y_test)))
y_test_pred = grid_search_lr.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_lr.best_params_))

CPU times: total: 41.5 s
Wall time: 5min 20s
accuracy на тестовом наборе: 0.74
f1_score на тестовом наборе: 0.77
Наилучшие значения гиперпараметров: {'C': 0.5, 'penalty': 'l1', 'solver': 'saga'}


In [74]:
#Случайный лес
param_grid_rf = {
                'n_estimators': list(range(50, 200, 25)), #Количество деревьев
                'min_samples_leaf': list(range(2, 10, 2)), # Количество листьев
                'max_depth': list(range(10, 30, 5)) # Максимальная высота дерева
               }

grid_search_rf = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(
        random_state=42, #генератор случайных чисел
    ), 
    param_grid=param_grid_rf, 
    cv=5, 
    n_jobs = -1
)  

%time grid_search_rf.fit(X_train, y_train) 
print("accuracy на тестовом наборе: {:.2f}".format(grid_search_rf.score(X_test, y_test)))
y_test_pred = grid_search_rf.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_rf.best_params_))

CPU times: total: 1.8 s
Wall time: 38.2 s
accuracy на тестовом наборе: 0.79
f1_score на тестовом наборе: 0.81
Наилучшие значения гиперпараметров: {'max_depth': 25, 'min_samples_leaf': 2, 'n_estimators': 75}


## <center> Random Search

In [75]:
#Линейная регрессия

random_search_lr = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42),
    param_distributions=param_grid_lr,
    cv=5,
    n_iter=20,
    n_jobs=-1
)

%time random_search_lr.fit(X_train, y_train) 
y_train_pred = random_search_lr.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(random_search_lr.score(X_test, y_test)))
y_test_pred = random_search_lr.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_lr.best_params_))

CPU times: total: 3.72 s
Wall time: 20.8 s
f1_score на обучающем наборе: 0.85
accuracy на тестовом наборе: 0.75
f1_score на тестовом наборе: 0.78
Наилучшие значения гиперпараметров: {'solver': 'saga', 'penalty': 'l1', 'C': 0.5}


In [76]:
#Случайный лес
random_search_rf = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42),
    param_distributions=param_grid_rf,
    cv=5,
    n_iter=20,
    n_jobs=-1
)

%time random_search_rf.fit(X_train, y_train) 
y_train_pred = random_search_rf.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(random_search_rf.score(X_test, y_test)))
y_test_pred = random_search_rf.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_rf.best_params_))

CPU times: total: 1.23 s
Wall time: 9.7 s
f1_score на обучающем наборе: 0.98
accuracy на тестовом наборе: 0.79
f1_score на тестовом наборе: 0.82
Наилучшие значения гиперпараметров: {'n_estimators': 125, 'min_samples_leaf': 2, 'max_depth': 15}


## <center> Hyperopt

In [77]:
# зададим пространство поиска гиперпараметров
# Параметры для линейной регрессии
space_lr_1={
    'penalty': hp.choice('penalty', ['l2', None]),
    'solver': hp.choice('solver', ['lbfgs', 'sag']),
    'C': hp.quniform('C', 0.01, 1, 0.1)
}
space_lr_2={
    'penalty': hp.choice('penalty', ['l1', 'l2']),
    'solver': hp.choice('solver', ['saga', 'liblinear']),
    'C': hp.quniform('C', 0.01, 1, 0.1)
}

#Параметры для случайного леса
space_rf={'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
       'max_depth' : hp.quniform('max_depth', 15, 26, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1)
      }

random_state=42
max_iter=1200

In [78]:
# Логистическая регрессия
def hyperopt_lr(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'penalty': params['penalty'], 
              'solver': str(params['solver']), 
              'C': float(params['C'])
              }
  
    # используем эту комбинацию для построения модели
    model = linear_model.LogisticRegression(**params, max_iter=max_iter, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return {'loss': -score, 'params': params, 'status': STATUS_OK}



# Случайный лес
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [79]:
%%time
# Подбор параметров для логистической регрессии
# Для первой подборки алгоритмов оптимизации

trials = Trials() # используется для логирования результатов

best_lr_1=fmin(hyperopt_lr, # наша функция 
          space=space_lr_1, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best_lr_1))
params_lr_1 = trials.best_trial['result']['params']

100%|██████████| 20/20 [07:10<00:00, 21.54s/trial, best loss: -0.7918133621344337]
Наилучшие значения гиперпараметров {'C': 0.1, 'penalty': 0, 'solver': 0}
CPU times: total: 3min 13s
Wall time: 7min 10s


In [80]:
%%time
# Для второй подборки алгоритмов оптимизации

trials = Trials() # используется для логирования результатов

best_lr_2=fmin(hyperopt_lr, # наша функция 
          space=space_lr_2, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best_lr_2))
params_lr_2 = trials.best_trial['result']['params']

100%|██████████| 20/20 [07:20<00:00, 22.02s/trial, best loss: -0.7955921602833411]
Наилучшие значения гиперпараметров {'C': 0.5, 'penalty': 0, 'solver': 1}
CPU times: total: 3min 16s
Wall time: 7min 20s


In [81]:
%%time
#Подбор параметров для случайного леса

trials = Trials() # используется для логирования результатов

best_rf=fmin(hyperopt_rf, # наша функция 
          space=space_rf, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров для случайного леса {}".format(best_rf))

100%|██████████| 20/20 [00:43<00:00,  2.15s/trial, best loss: -0.8132767563021502]
Наилучшие значения гиперпараметров для случайного леса {'max_depth': 18.0, 'min_samples_leaf': 2.0, 'n_estimators': 103.0}
CPU times: total: 18.8 s
Wall time: 43.1 s


In [82]:
# рассчитаем точность для тестовой выборки
# Логистическая регрессия 1
model = linear_model.LogisticRegression(
    random_state=random_state,
    max_iter=max_iter,
    penalty=str(params_lr_1['penalty']),
    solver=str(params_lr_1['solver']),
    C=float(params_lr_1['C'])
)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))


print('-'*50)
# Логистическая регрессия 2
model = linear_model.LogisticRegression(
    random_state=random_state,
    max_iter=max_iter,
    penalty=str(params_lr_2['penalty']),
    solver=str(params_lr_2['solver']),
    C=float(params_lr_2['C'])
)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))


print('-'*100)
# Случайный лес
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best_rf['n_estimators']),
    max_depth=int(best_rf['max_depth']),
    min_samples_leaf=int(best_rf['min_samples_leaf'])
)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на обучающем наборе: 0.85
accuracy на тестовом наборе: 0.74
f1_score на тестовом наборе: 0.77
--------------------------------------------------
f1_score на обучающем наборе: 0.86
accuracy на тестовом наборе: 0.74
f1_score на тестовом наборе: 0.77
----------------------------------------------------------------------------------------------------
f1_score на обучающем наборе: 0.99
accuracy на тестовом наборе: 0.79
f1_score на тестовом наборе: 0.82


## <center> Optuna

In [83]:
# Функции для логистических регрессий
# 1 набор параметров
def optuna_lr_1(trial, cv=5, X=X_train, y=y_train):
  # задаем пространства поиска гиперпараметров
  penalty = trial.suggest_categorical('penalty', ['l2', None])
  solver = trial.suggest_categorical('solver', ['lbfgs', 'sag'])
  C = trial.suggest_float('C', 0.01, 1, step=0.09)

  # создаем модель
  model = linear_model.LogisticRegression(penalty=penalty,
                                          solver=solver,
                                          C=C,
                                          random_state=random_state,
                                          max_iter=max_iter
                                          )
  # обучаем модель
  model.fit(X_train, y_train)
  score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

  return score

# 2 набор параметров
def optuna_lr_2(trial, cv=5, X=X_train, y=y_train):
  # задаем пространства поиска гиперпараметров
  penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
  solver = trial.suggest_categorical('solver', ['saga', 'liblinear'])
  C = trial.suggest_float('C', 0.01, 1, step=0.09)

  # создаем модель
  model = linear_model.LogisticRegression(penalty=penalty,
                                          solver=solver,
                                          C=C,
                                          random_state=random_state,
                                          max_iter=max_iter
                                          )
  # обучаем модель
  model.fit(X_train, y_train)
  score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

  return score


#Для случайного леса
def optuna_rf(trial, cv=5, X=X_train, y=y_train):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
  max_depth = trial.suggest_int('max_depth', 15, 26, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state
                                          )
  # обучаем модель
  model.fit(X_train, y_train)
  score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

  return score

In [84]:
%%time
# cоздаем объект исследования
# Логистическая регрессия с набором параметров 1
study_lr_1 = optuna.create_study(study_name="Logistic_Regression_1", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study_lr_1.optimize(optuna_lr_1, n_trials=20)

[I 2025-02-05 12:03:48,036] A new study created in memory with name: Logistic_Regression_1
[I 2025-02-05 12:03:48,810] Trial 0 finished with value: 0.7918133621344337 and parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.09999999999999999}. Best is trial 0 with value: 0.7918133621344337.
[I 2025-02-05 12:04:40,603] Trial 1 finished with value: 0.7579718891118589 and parameters: {'penalty': None, 'solver': 'sag', 'C': 0.19}. Best is trial 0 with value: 0.7918133621344337.
[I 2025-02-05 12:05:29,627] Trial 2 finished with value: 0.7579718891118589 and parameters: {'penalty': None, 'solver': 'sag', 'C': 0.28}. Best is trial 0 with value: 0.7918133621344337.
[I 2025-02-05 12:05:49,019] Trial 3 finished with value: 0.7903853579022508 and parameters: {'penalty': 'l2', 'solver': 'sag', 'C': 0.19}. Best is trial 0 with value: 0.7918133621344337.
[I 2025-02-05 12:05:53,070] Trial 4 finished with value: 0.7274448316780647 and parameters: {'penalty': None, 'solver': 'lbfgs', 'C': 0.64}. Bes

CPU times: total: 2min 34s
Wall time: 5min 3s


In [85]:
%%time
# cоздаем объект исследования
# Логистическая регрессия с набором параметров 2
study_lr_2 = optuna.create_study(study_name="Logistic_Regression_2", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study_lr_2.optimize(optuna_lr_2, n_trials=20)

[I 2025-02-05 12:08:51,831] A new study created in memory with name: Logistic_Regression_2
[I 2025-02-05 12:10:02,964] Trial 0 finished with value: 0.7927096999769719 and parameters: {'penalty': 'l1', 'solver': 'saga', 'C': 0.28}. Best is trial 0 with value: 0.7927096999769719.
[I 2025-02-05 12:10:03,543] Trial 1 finished with value: 0.7763809622952911 and parameters: {'penalty': 'l2', 'solver': 'liblinear', 'C': 1.0}. Best is trial 0 with value: 0.7927096999769719.
[I 2025-02-05 12:11:18,678] Trial 2 finished with value: 0.7923519422881288 and parameters: {'penalty': 'l1', 'solver': 'saga', 'C': 0.73}. Best is trial 0 with value: 0.7927096999769719.
[I 2025-02-05 12:11:19,101] Trial 3 finished with value: 0.7912284304278898 and parameters: {'penalty': 'l2', 'solver': 'liblinear', 'C': 0.09999999999999999}. Best is trial 0 with value: 0.7927096999769719.
[I 2025-02-05 12:12:44,195] Trial 4 finished with value: 0.7972413946352239 and parameters: {'penalty': 'l1', 'solver': 'saga', 'C': 

CPU times: total: 8min 9s
Wall time: 17min 27s


In [86]:
%%time
# cоздаем объект исследования
# Случайный лес
study_rf = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study_rf.optimize(optuna_rf, n_trials=20)

[I 2025-02-05 12:26:19,223] A new study created in memory with name: RandomForestClassifier
[I 2025-02-05 12:26:22,035] Trial 0 finished with value: 0.8064827273406159 and parameters: {'n_estimators': 176, 'max_depth': 17, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.8064827273406159.
[I 2025-02-05 12:26:25,007] Trial 1 finished with value: 0.8078129193338942 and parameters: {'n_estimators': 193, 'max_depth': 23, 'min_samples_leaf': 6}. Best is trial 1 with value: 0.8078129193338942.
[I 2025-02-05 12:26:31,519] Trial 2 finished with value: 0.811207334137003 and parameters: {'n_estimators': 195, 'max_depth': 16, 'min_samples_leaf': 3}. Best is trial 2 with value: 0.811207334137003.
[I 2025-02-05 12:26:33,740] Trial 3 finished with value: 0.8081181453204558 and parameters: {'n_estimators': 130, 'max_depth': 21, 'min_samples_leaf': 4}. Best is trial 2 with value: 0.811207334137003.
[I 2025-02-05 12:26:35,632] Trial 4 finished with value: 0.8050631157567372 and parameters: {'n_est

CPU times: total: 20.9 s
Wall time: 49 s


In [87]:
# выводим результаты на обучающей выборке
# Логистическая регрессия 1
print("Наилучшие значения гиперпараметров {}".format(study_lr_1.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study_lr_1.best_value))

# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(**study_lr_1.best_params, random_state=random_state, max_iter=max_iter)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

#Визуализируем оптимизацию
optuna.visualization.plot_optimization_history(study_lr_1, target_name="f1_score")

Наилучшие значения гиперпараметров {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.09999999999999999}
f1_score на обучающем наборе: 0.79
accuracy на тестовом наборе: 0.74
f1_score на тестовом наборе: 0.77


In [88]:
# выводим результаты на обучающей выборке
# Логистическая регрессия 2
print("Наилучшие значения гиперпараметров {}".format(study_lr_2.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study_lr_2.best_value))

# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(**study_lr_2.best_params, random_state=random_state, max_iter=max_iter)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

#Визуализируем оптимизацию
optuna.visualization.plot_optimization_history(study_lr_2, target_name="f1_score")

Наилучшие значения гиперпараметров {'penalty': 'l1', 'solver': 'saga', 'C': 0.45999999999999996}
f1_score на обучающем наборе: 0.80
accuracy на тестовом наборе: 0.75
f1_score на тестовом наборе: 0.77


In [89]:
# выводим результаты на обучающей выборке
# Случайный лес
print("Наилучшие значения гиперпараметров {}".format(study_rf.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study_rf.best_value))

# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study_rf.best_params,random_state=random_state, )
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

#Визуализируем оптимизацию
optuna.visualization.plot_optimization_history(study_rf, target_name="f1_score")

Наилучшие значения гиперпараметров {'n_estimators': 142, 'max_depth': 18, 'min_samples_leaf': 3}
f1_score на обучающем наборе: 0.81
accuracy на тестовом наборе: 0.79
f1_score на тестовом наборе: 0.81


## <center> Выводы

Лучшая модель по метрикам - Случайный лес, оптимизированный с помощью Optuna. Самая лучшая связка же среди регрессий - 1-я связка, оптимизированная с помощью Hyperopt, но здесь оптимизация играет роль погрешности - разница по сравнению с базовыми моделями спорная. И ненужная в остальных случаях, так как абсолютно идентичные (если округление оставить до 2 знака после запятой). Со случайным лесом та же история, в целом. То есть метрики или остались такими же, либо улучшились (или ухудшились в некоторых редких случаях) на 0.01. Возможно, стоит поднять количество итераций.

По удобству работы - лучше всего показала себя Optuna. При возможно использовать традиционные GridSearch и RandomSearch в некоторых сценариях.

P.S. Кросс-валидация использована только при обучении в процессе подбора гиперпараметров. В задании не уточняется, везде ли она должна быть использована, а хотелось бы.